# Decision Tree Algorithm for CMV Delta Winners

We will create a basic decision tree with the first decision rules to test:
* Words of Certainty
* Words of Extremity
* Lexical Diversity
* Total Number of Characters
* Number of Provided Links
* Whether or not OP was quoted

#### TODO
~~Decide how to calculate decision rules<br>~~
~~Figure out what spans to use for:<br>~~
 * ~~Lexical Diversity~~
 * ~~Number of Chars~~
 * ~~Number of links~~

Write func to clean body<br>
Fix getNumQuotes()<br>
Optimize Functions by limiting # of loops used on same file<br>
Write values for decision tree into CSV file<br>
Learn to read and write from csv file using pandas<br>

 
#### Notes
Given CSV files are encoded as UTF-8

In [84]:
#optimize the loops so you only need to run through the text once

# these are the words of certainty
wOC = ["absolutely", "always", "certain", "commit", "completely", "every", "exact"] 

# this is a function that counts the number of words of certainty
getCertaintyCount = lambda text,word_i=0: text.count(wOC[word_i]) if word_i == len(wOC) - 1 else text.count(wOC[word_i]) + getCertaintyCount(text, word_i+1)


# initial words of extremity
eWC = ["much", "more", "extremely", "very", "wonderful"]

# this function counts the total of 
getExtremityCount = lambda text,word_i=0: text.count(eWC[word_i]) if word_i == len(eWC) - 1 else text.count(eWC[word_i]) + getExtremityCount(text, word_i+1)



# here is the calculator for lexical diversity
def getLexicalDiversity(text):
    lstOfWords = text.split()
    setOfWords = set(lstOfWords)
    return len(setOfWords)/len(lstOfWords)


# here is the counter for number of links
def getNumLinks(text):
    return text.count("<a")


# here is the counter for num of quotes
def getNumQuotes(text):
    return text.count("\n> ")



This will test the functions created above.

In [85]:
text = "i absolutely very very much love using words like certain commit and wonderful here is a link http://google.com. \n>This person said this"
print(f"Number of words of certainty: {getCertaintyCount(text)}")
print(f"Number of words of extremity: {getExtremityCount(text)}")
print(f"Lexical Diversity: {getLexicalDiversity(text)}")
print(f"Number of links: {getNumLinks(text)}")
print(f"Number of quotes: {getNumQuotes(text)}")

Number of words of certainty: 3
Number of words of extremity: 4
Lexical Diversity: 0.9545454545454546
Number of links: 0
Number of quotes: 0


### Opening and Testing the CSV FIle uing CSV Module

We start here by attempting to open the file and read a certain amount of lines(3 lines).
This is the basic setup as to how we're going to read through the CSV files.

In [78]:
import csv
lines_to_read = 3
with open('TextData.csv', mode='r', encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}\n')
            line_count += 1
        print(f'\tAuthor:\n{row["author"]}\n\tInitial Content:\n{row["body"][0:100]}.\n')
        line_count += 1
        if line_count >= lines_to_read:
              break;
    print(f'Processed {line_count} lines.')

Column names are subreddit, created_utc, author, id, parent_id, body, body_html, Delta_Awarded, Delta_Awarder, Delta_AwardText, Submission_Author

	Author:
Thompson_S_Sweetback
	Initial Content:
1. Long term economic disincentives will not be very effective because people are naturally optimist.

1358441654
	Author:
llatia
	Initial Content:
I think most people would say that whether they act on desires and thoughts or not is precisely the .

1358460613
Processed 3 lines.


### Determining the span for Lexical Diversity and Number of Characters

Manipulating data gets boring if that's all you do, so I'll start by creating a representation of both from the first 100 comments. 

<font color=blue>***Let's have some fun!***</font>

* Will multiply Lexical Diversity by 100, then round to the nearest 10
* Will round number of characters to the nearest 1000

The visual representation will be the rounded numbers by the amount of occurences of those numbers

We will start by filling in the dictionaries lexicalDiversity and numOfChars with the required data.

In [79]:
import csv
lines_to_read = 1000

lexicalDiversity = {}
numOfChars = {}

with open('TextData.csv', mode='r', encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}\n')
            line_count += 1
                  
        # file manipulation here
        #use row[column_name] to get data within that column
                  
        #this shows the first 100 chars of data in the comments
        # print(f'\tAuthor:\n{row["author"]}\n\tInitial Content:\n{row["body"][0:100]}.\n') 
                  
        text_length = len(row["body"])
        text_length = round(text_length, -3)
        if text_length not in numOfChars.keys():
              numOfChars[text_length] = 1
        else:
              numOfChars[text_length] += 1
        
        cur_lex_div = getLexicalDiversity(row["body"])
        cur_lex_div = round(100 * cur_lex_div, -1)
        if cur_lex_div not in lexicalDiversity.keys():
              lexicalDiversity[cur_lex_div] = 1
        else:
              lexicalDiversity[cur_lex_div] += 1
              
        #file manipulation ends here
        line_count += 1
        if line_count >= lines_to_read:
              break;
        #print(line_count) 
    print("Rounded number of characters: Occurences")
    
    result = ""
    for x,y in sorted(numOfChars.items()):
        result += f"{x}: {y} | "
    print(result[:-2],"\n")
    #print(numOfChars,"\n")
    
    print("Rounded lexical diversities: Occurences")
    result = ""
    for x,y in sorted(lexicalDiversity.items()):
        result += f"{x}: {y} | "
    print(result[:-2],"\n")
        
    print(f'Processed {line_count} lines.')

Column names are subreddit, created_utc, author, id, parent_id, body, body_html, Delta_Awarded, Delta_Awarder, Delta_AwardText, Submission_Author

Rounded number of characters: Occurences
0: 506 | 1000: 334 | 2000: 87 | 3000: 41 | 4000: 18 | 5000: 3 | 6000: 4 | 7000: 2 | 8000: 2 | 9000: 1 | 10000: 1  

Rounded lexical diversities: Occurences
40.0: 6 | 50.0: 41 | 60.0: 168 | 70.0: 227 | 80.0: 220 | 90.0: 186 | 100.0: 151  

Processed 1000 lines.


### Time to write some data to a csv file...

We're going to first write our headers to our first row, which will be:
(subreddit, author, parent_id, id, Delta_Awarded, body, body_html, certainty_count, extremity_count, 
    lexical_diversity_rounded, char_count_rounded, link_count, quote_count)

Then we're gonna go through and fill in our data!
I'll write to a file called "OutputTest.csv" for now.

In [100]:
import csv
write_file_name = "OutputTest.csv"
file = open(write_file_name, mode="w")
writer = csv.writer(file, delimiter=',', quotechar='"')

#first writer header row
writer.writerow(['subreddit', 'author', 'parent_id', 'id', 'Delta_Awarded', 'body', 'body_html', 'certainty_count', 'extremity_count', 
    'lexical_diversity_rounded', 'char_count_rounded', 'link_count', 'quote_count'])

print("Header row written.")

lines_to_read = 3
with open('TextData.csv', mode='r', encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Initial columns being read: {", ".join(row)}\n')
            line_count += 1
        
        
        
        #calculate certainty_count, extremity_count, lexical_diversity_rounded, char_count_rounded, link_count, quote_count
                  
        certainty_count = getCertaintyCount(row['body'])
        extremity_count = getExtremityCount(row['body'])
        lexical_diversity = getLexicalDiversity(row["body"])
        lexical_diversity_rounded = round(100 * lexical_diversity, -1)
        char_count_rounded = round(len(row["body"]), -3)
        link_count = getNumLinks(row["body_html"])
        quote_count = getNumQuotes(row["body"])
                  
        #column order: (subreddit, author, parent_id, id, Delta_Awarded, body, body_html, certainty_count, extremity_count, 
        #lexical_diversity_rounded, char_count_rounded, link_count, quote_count)
        writer.writerow([row['subreddit'], row['author'], row['parent_id'], row['id'], row['Delta_Awarded'], row['body'], row['body_html'],
                                  certainty_count, extremity_count, lexical_diversity_rounded, char_count_rounded, link_count, quote_count])
                  
        
        line_count += 1
        if line_count >= lines_to_read:
              break;
    print(f'Processed {line_count} lines.')
file.close()


Header row written.
Initial columns being read: subreddit, created_utc, author, id, parent_id, body, body_html, Delta_Awarded, Delta_Awarder, Delta_AwardText, Submission_Author

Processed 3 lines.
